# Benchmarking de transposição de eixos
Este notebook compara o desempenho de diferentes implementações de transposição de eixos.

**Nota:** os resultados do benchmarking variam muito dependendo do tamanho da imagem, tamanho do kernel, operações utilizadas, parâmetros e hardware utilizado. Use este notebook para adaptá-lo ao seu cenário de uso e faça o benchmark no seu hardware alvo. Se você tiver cenários ou casos de uso diferentes, você é muito bem-vindo para enviar seu notebook como pull-request!

In [1]:
import pyclesperanto_prototype as cle
import numpy as np
import time
import cupy as cp

# para medir a duração da execução do kernel corretamente, precisamos definir esta flag. Isso irá diminuir um pouco a execução dos fluxos de trabalho
cle.set_wait_for_kernel_finish(True)

# selecione uma GPU com o seguinte nome. Isso irá recorrer a qualquer outra GPU se nenhuma com esse nome for encontrada
cle.select_device('RTX')

<NVIDIA GeForce RTX 3050 Ti Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# dados de teste
import numpy as np

test_image = np.random.random([100, 512, 1024])

## clEsperanto

In [3]:
# transposição com pyclesperanto
result_image = None

test_image_gpu = cle.push_zyx(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.transpose_xz(test_image_gpu, result_image)
    print("duração da transposição pyclesperanto: " + str(time.time() - start_time))
print(result_image.shape)

pyclesperanto transpose duration: 0.06201291084289551
pyclesperanto transpose duration: 0.04100918769836426
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.0400090217590332
pyclesperanto transpose duration: 0.04129624366760254
pyclesperanto transpose duration: 0.042009592056274414
pyclesperanto transpose duration: 0.04128861427307129
pyclesperanto transpose duration: 0.04102063179016113
pyclesperanto transpose duration: 0.04099869728088379
(1024, 512, 100)


## cupy

In [4]:
# transposição com numpy
result_image = None
cu_test_image = cp.asarray(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cp.transpose(cu_test_image, (2, 1, 0))
    cp.cuda.stream.get_current_stream().synchronize() # precisamos esperar aqui para medir o tempo corretamente
    print("duração da transposição cupy: " + str(time.time() - start_time))
print(result_image.shape)

cupy transpose duration: 0.06301379203796387
cupy transpose duration: 0.0009999275207519531
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
(1024, 512, 100)


## numpy

In [5]:
# transposição com numpy
result_image = None

for i in range(0, 10):
    start_time = time.time()
    result_image = np.transpose(test_image, (2, 1, 0))
    print("duração da transposição numpy: " + str(time.time() - start_time))
print(result_image.shape)

numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
(1024, 512, 100)
